In [1]:
from web_parser import AnswerParser, QuestionParser
import multiprocessing
from multiprocessing import Queue, Process, Event
from selenium import webdriver
from pathlib import Path
import os
import yaml
from config import drivers, nr_answer_parsers, get_db_path, get_vcs_path, get_config_path
from data_handler import Master, init_new_repo, update_database
from util.logger import Logger
import time

In [2]:
# 需要爬取的问题的列表
q_list = [
    # 623539562,
    # 327003589
    # 624316466
    # 524257471,
    # 437735833
    # 2420199852
    551585913
]
# 存档仓库的地址 # 请修改为您的地址
archiv_path = os.path.join(
    "/media/ywatcher/ExtDisk1/Files/web_archiv_",
    "archiv_2"
)
# archiv_path = "your/path"

In [3]:
# if you have an old version repo
update_database(repo_path=archiv_path)

2.6.0


In [4]:
# 初始化存档仓库，如果存档路径已经存在且不是文件夹，
# 或者存档路径已经存在或者非空，
# 或者存档路径的上一级目录不存在/不是文件夹，
# 会抛出异常，并终止创建仓库。
try:
    init_new_repo(
        repo_path=archiv_path,
    )
except Exception as e:
    print(e)

/media/ywatcher/ExtDisk1/Files/web_archiv_/archiv_2 exists, and is not empty


In [5]:
# 初始化进程间交流使用的队列
q_task = Queue()
q_result = Queue()

如果一个进程 `p` 使用进程队列 `q` 并往里面放入数据，
那么当队列 `q` 在读入大量数据之后，需要将 `q` 中大部
分数据取出后，进程 `p` 才能执行 `.join()`；否则可能
会在 `join()` 时阻塞，此时内部执行的函数是等待 `q` 
拿到它的线程锁（队列 `q` 维护了一个它自己的线程）.<br>
https://stackoverflow.com/questions/71947784/what-does-webdriverwaitdriver-20-mean<br>
https://stackoverflow.com/questions/26869200/python-queue-module-get-stuck



In [6]:
# 根据仓库路径，初始化各个子部分的路径，并读取仓库配置文件
db_path = get_db_path(archiv_repo_path=archiv_path)
vcs_path = get_vcs_path(archiv_repo_path=archiv_path)
repo_config_path = get_config_path(archiv_repo_path=archiv_path)
repo_config = yaml.safe_load(open(repo_config_path, 'r')) 

In [7]:
repo_config

{'owner_email': 'me@email', 'owner_name': 'me'}

In [8]:
# 初始化 selenium web driver 和爬取器
# 初始化 web driver 时，会有相应的窗口
# 打开
# 初始化问题爬取器
question_driver = webdriver.Chrome(drivers["chrome"])
question_parser = QuestionParser(
    queue_put_result=q_result,
    queue_put_task=q_task,
    driver=question_driver,
    logger=Logger("q")
)

# 初始化多个回答爬取器
answer_drivers = []
answer_parsers = []
for i in range(nr_answer_parsers):
    answer_drivers.append(
        webdriver.Chrome(drivers["chrome"])
    )
    answer_parsers.append(
        AnswerParser(
            queue_put_result=q_result,
            queue_get_task=q_task,
            driver=answer_drivers[i],
            logger=Logger("a_{}".format(i))
        )
    )
fin_events = [
    question_parser.fin_event
] + [
    answer_parser.fin_event
    for answer_parser in answer_parsers
]
            
# 初始化爬取结果处理器
master = Master(
    db_path=db_path,
    git_repo_path=vcs_path,
    logger=Logger("m")
)


In [9]:
# 初始化爬取器和处理器的进程
targets = [
    # target func and args
    (question_parser.start_parsing_list, (q_list,))
] + [
    (answer_parser.start_parsing, (question_parser.fin_event,))
    for answer_parser in answer_parsers
] + [
    # (master.start, (config["commiter_name"],config["commiter_email"],q_result))
    (master.start_parse, (
        repo_config['owner_name'],
        repo_config["owner_email"],
        q_result,
        fin_events
    ))
]


processes = [
    Process(target=target_func, args=args)
    for target_func, args in targets
]







In [10]:
# 开启爬取进程
for p in processes[:-1]:
# for p in processes:
    p.start()

# 目前有未解决的进程死锁问题，因此通过先运行所有 parser
# 进程, 当所有 parser 的运行完成 flag 设为 True 后
# 再运行档案管理进程。
while not all([e.is_set() for e in fin_events]):
    time.sleep(5)
processes[-1].start()
    
for p in processes:
    p.join()

551585913
q: parsing question with id=551585913
q: Scrolled to bottom.
q: a_0:parsed 551585913 
parsing answer with id=2658341990q:
 fin
a_1: stop
a_1: puta_2:
 a_1:stop 
parsing done.
a_2: put
a_2: parsing done.
a_1: out
a_2: out
a_0: stop
a_0: put
a_0: parsing done.
a_0: out
m: get question: 551585913
m: question 551585913 is in database
m: get answer: 2658341990
m: answer 2658341990 is in database
m: version update
m: done.


In [ ]:
class Controller:
    def __init__(
        self, 
    ):
        self.processes = []
        pass